---
toc: true
comments: false
layout: post
title: RDS
description: This blog is for RDS info and setup
type: hacks
courses: { csa: {week: 30} }
---

### Understanding AWS RDS (Relational Database Service)

**Amazon RDS (Relational Database Service)** is a managed database service that allows you to set up, operate, and scale a relational database in the cloud easily. It provides scalable capacity while managing time-consuming database administration tasks such as hardware provisioning, database setup, patching, and backups. It offers high availability and security options, including automated backups, database snapshots, and automatic host replacement.

### MySQL vs SQLite on RDS

When you create a database instance in RDS, you choose a database engine. RDS supports several engines such as MySQL, PostgreSQL, Oracle, SQL Server, and MariaDB. The choice of engine determines the features and the SQL dialect that you'll use. For instance:
- **MySQL on RDS**: If you choose MySQL when setting up your RDS instance, your database hosted on AWS will be using the MySQL engine. It means that your application needs to be compatible with MySQL SQL syntax and configurations.
- **SQLite**: This is a lightweight, file-based database, different from MySQL. SQLite is typically used for development, testing, or small applications given its simplicity and the fact that it doesn't require a separate server process.

### Switching Databases from SQLite to MySQL

Your question about whether your backend needs to switch from SQLite to MySQL depends on how you set up your application's database configurations. In practice, it's quite common to use SQLite for local development due to its simplicity and then switch to a more scalable option like MySQL in production.

### Configuring Spring Boot for Different Environments

In Spring Boot, you can manage different configurations for different environments using profile-specific application properties files. Here’s how you can set this up:

1. **application.properties**: This will be your main configuration file.
2. **application-dev.properties**: This will hold configurations for your development environment (using SQLite).
3. **application-prod.properties**: This will hold configurations for your production environment (using RDS MySQL).

#### Example Configuration

For development (`application-dev.properties`):

```properties
spring.datasource.url=jdbc:sqlite:path_to_your_local_db_file.db
spring.datasource.driver-class-name=org.sqlite.JDBC
spring.jpa.database-platform=org.hibernate.dialect.SQLiteDialect
```

For production (`application-prod.properties`):

```properties
spring.datasource.url=jdbc:mysql://your-rds-instance-endpoint:3306/yourdatabase
spring.datasource.username=your_username
spring.datasource.password=your_password
spring.datasource.driver-class-name=com.mysql.cj.jdbc.Driver
spring.jpa.database-platform=org.hibernate.dialect.MySQL5Dialect
spring.jpa.hibernate.ddl-auto=update
```

### Running Spring Boot with Profiles

To activate a specific profile, you can run your Spring Boot application with an environment variable:

```sh
java -jar -Dspring.profiles.active=dev your-app.jar   # For development
java -jar -Dspring.profiles.active=prod your-app.jar  # For production
```

Or, if you're using Maven or Gradle to run locally:

```sh
mvn spring-boot:run -Dspring-boot.run.profiles=dev
```

This setup allows your application to seamlessly transition from using SQLite in development to using MySQL in AWS RDS in production, keeping local development fast and simple while leveraging more robust and scalable database management features in production.